In [3]:
import numpy as np
from IPython.display import display

np.set_printoptions(suppress=True)

#np.zeros(5)
sum([1,2,3,4])
display('{} {}'.format('one', 'two'))

'one two'

In [4]:
class HMM:
    def __init__(self, hidden_states, visible_symbols):
        
        # first hidden state is actually w0 (omega_0)
        self.hidden_states = ["w0"] + hidden_states
        # first visible symbol is actually v0
        self.visible_symbols = ["v0"] + visible_symbols
    
        self.count_hidden = len(hidden_states) + 1
        self.count_visible = len(visible_symbols) + 1
    
        # transition matrix
        self.a = np.random.random(self.count_hidden ** 2).reshape(self.count_hidden, self.count_hidden)
        # emissions matrix
        self.b = np.random.random(self.count_hidden * self.count_visible).reshape(self.count_hidden, self.count_visible)
        # alpha matrix
        #self.alpha_matrix = np.zeros(self.count_hidden ** 2).reshape(self.count_hidden, self.count_hidden)
        # beta matrix
        self._beta = np.zeros(self.count_hidden ** 2).reshape(self.count_hidden, self.count_hidden)
        
        self.a[0, 0] = 1
        for i in range(1, self.count_hidden):
            self.a[0, i] = 0
            
        self.b[0, 0] = 1
        for k in range(1, self.count_visible):
            self.b[0, k] = 0
        
        for j in range(1, self.count_hidden):
            self.b[j, 0] = 0
    
    def alpha(self, j, t, sequence, initial_hidden_state = 1, force_refresh = False):
        if force_refresh:
            self.alpha_matrix = np.zeros(self.count_hidden * len(sequence)).reshape(self.count_hidden,  len(sequence))
            display("Force refresh")
        else:
            #display("getting precalculated value")
            return self.alpha_matrix[j, t]
        
        for t in range(len(sequence)):
            k = self.visible_symbols.index(sequence[t])
            for j in range(self.count_hidden):
                if t == 0 and j != initial_hidden_state:
                    self.alpha_matrix[j, t] = 0
                elif t == 0 and j == initial_hidden_state:
                    self.alpha_matrix[j, t] = 1
                else:
                    display('j:{} t:{} k:{}, vk:{}'.format(j, t, k, sequence[t]))
                    suma = 0
                    for i in range(self.count_hidden):
                        display('alpha_matrix_i{}_t{}:{} ai{}j{}:{}'.format(i, t-1, self.alpha_matrix[i, t-1], i, j, self.a[i,j]))
                        suma += self.alpha_matrix[i, t-1] * self.a[i, j] * self.b[j, k] 
                    
                    display("bj{}k{}:{} result:{}".format(j, k,self.b[j,k], suma))
                    self.alpha_matrix[j, t] =  suma 

#model = HMM(["w1", "w2", "w3"], ["v1", "v2", "v3", "v4"])

#model.alpha(0, 4 , ["v3", "v1", "v2", "v1", "v0"], force_refresh = True)
#model.alpha_matrix

In [5]:
model2 = HMM(["w1", "w2", "w3"], ["v1", "v2", "v3", "v4"])


#                 W0   W1   W2    W3
aij = np.array([(  1,   0,   0,   0),  # W0
                (0.2, 0.3, 0.1, 0.4),  # W1
                (0.2, 0.5, 0.2, 0.1),  # W2
                (0.7, 0.1, 0.1, 0.1)]) # W3


#                 V0   V1   V2   V3   V4
bjk = np.array([(  1,   0,   0,   0,   0),   # W0
                (  0, 0.3, 0.4, 0.1, 0.2),   # W1
                (  0, 0.1, 0.1, 0.7, 0.1),   # W2
                (  0, 0.5, 0.2, 0.1, 0.2)])  # W3

model2.a = aij
model2.b = bjk

model2.alpha(0, 4 , ["v0", "v1", "v3", "v2", "v0"], force_refresh = True)
model2.alpha_matrix

'Force refresh'

'j:0 t:1 k:1, vk:v1'

'alpha_matrix_i0_t0:0.0 ai0j0:1.0'

'alpha_matrix_i1_t0:1.0 ai1j0:0.2'

'alpha_matrix_i2_t0:0.0 ai2j0:0.2'

...

'alpha_matrix_i1_t3:0.005192 ai1j3:0.4'

'alpha_matrix_i2_t3:0.000543 ai2j3:0.1'

'alpha_matrix_i3_t3:0.000964 ai3j3:0.1'

'bj3k0:0.0 result:0.0'

array([[0.       , 0.       , 0.       , 0.       , 0.0018218],
       [1.       , 0.09     , 0.0052   , 0.005192 , 0.       ],
       [0.       , 0.01     , 0.0217   , 0.000543 , 0.       ],
       [0.       , 0.2      , 0.0057   , 0.000964 , 0.       ]])

In [6]:
model.a

NameError: name 'model' is not defined

In [7]:
model.b

NameError: name 'model' is not defined

In [8]:
0.09*0.3*0.1 + 0.01 * 0.5 * 0.1 + 0.2 * 0.1 * 0.1

0.005200000000000001

In [9]:
0.7 * 0.1 * 0.09 + 0.7 * 0.2 * 0.01 + 0.7 * 0.1 * 0.2

0.021699999999999997